In [1]:
import tensorflow as tf

In [2]:
import matplotlib.pylab as plt
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import itertools

In [3]:
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix

In [4]:
import os
import numpy as np
import glob
import shutil

In [5]:
BATCH_SIZE = 32
IMG_SHAPE = 224
class_names = ['Fire !!', 'No Fire']

In [6]:
train_path = 'MyDataset/train'
valid_path = 'MyDataset/validation'
test_path = 'MyDataset/test'

In [ ]:
# plots images with predictions within jupyter notebook
def plots(ims, figsize=(12,6), rows=1, interp=False, titles=None):
    if type(ims[0]) is np.ndarray:
        ims = np.array(ims).astype(np.uint8)
        if (ims.shape[-1] != 3):
            ims = ims.transpose((0,2,3,1))
    f = plt.figure(figsize=figsize)
    cols = len(ims)//rows if len(ims) % 2 == 0 else len(ims)//rows + 1
    for i in range(len(ims)):
        sp = f.add_subplot(rows, cols, i+1)
        sp.axis('Off')
        plt.xlabel("Actual : " + class_names[int(test_labels[i])])
        if titles is not None:
            #sp.set_title(titles[i], fontsize=16)
            sp.set_title(class_names[int(titles[i])])
        plt.imshow(ims[i], interpolation=None if interp else 'none')

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
image_gen_train = ImageDataGenerator(      
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

train_data_gen = image_gen_train.flow_from_directory(directory=train_path, 
                                                     target_size=(IMG_SHAPE,IMG_SHAPE),
                                                     shuffle=True,
                                                     classes=['Fire', 'NoFire'],
                                                     batch_size=BATCH_SIZE, 
                                                     class_mode='binary')

In [ ]:
image_gen_val = ImageDataGenerator(rescale=1./255)

val_data_gen = image_gen_val.flow_from_directory(batch_size=BATCH_SIZE,
                                                 directory=valid_path,
                                                 classes=['Fire', 'NoFire'],
                                                 target_size=(IMG_SHAPE,IMG_SHAPE),
                                                 class_mode='binary')

In [ ]:
URL = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
feature_extractor = hub.KerasLayer(URL,
                                   input_shape=(IMG_SHAPE, IMG_SHAPE,3))

In [ ]:
feature_extractor.trainable = False

In [ ]:
model = tf.keras.Sequential([
  feature_extractor,
  layers.Dense(2, activation='softmax')
])

In [ ]:
model.summary()

In [ ]:
model.compile(
  optimizer='adam',
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

EPOCHS = 5
history = model.fit(train_data_gen,
                    epochs=EPOCHS,
                    validation_data=val_data_gen)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(EPOCHS)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
img_gen = ImageDataGenerator()
test_batches = img_gen.flow_from_directory(test_path, target_size=(IMG_SHAPE,IMG_SHAPE), 
                                           classes=['Fire', 'NoFire'], 
                                           batch_size=10,
                                           shuffle=True,
                                           class_mode='binary')

In [ ]:
test_imgs, test_labels = next(test_batches)
predictions = model.predict_generator(test_batches, steps=1, verbose=0)

predictions_arr = []
for i in range(0,10):
    predictions_arr.append(np.argmax(predictions[i]))
    
plots(test_imgs, titles=test_labels)
plots(test_imgs, titles=predictions_arr)

In [ ]:
cm = confusion_matrix(test_labels, predictions_arr)
cm_plot_labels=['Fire','NoFire']
plot_confusion_matrix(cm,cm_plot_labels,title="Confusion Matrix")